# Visualizing the Quality of Life

In [78]:
import pandas as pd
import altair as alt

In [79]:
source = pd.read_excel('OECD_Data.xlsx')

# Convert the units row (row 1) into a dictionary
units = source.iloc[0].to_dict()

# Create a dictionary for simplified names of the units
simplified_units = {
    "Percentage": "%",
    "Ratio": "ratio",
    "US Dollar": "$",
    "Average score": "average score",
    "Years": "years",
    "Micrograms per cubic metre": "µg/m³",
    "Hours": "hours"
}

# Remove the units row from the source data
source = source.iloc[1:]

# Reset the DataFrame index
source.reset_index(drop=True, inplace=True)

# Add units to each of the indicators
for column in source.columns:
    unit = units[column]
    if unit in simplified_units:
        source.rename(columns={column: f"{column} ({simplified_units[unit]})"}, inplace=True)

source.head(10)

,Country,Dwellings without basic facilities (%),Housing expenditure (%),Rooms per person (ratio),Household net adjusted disposable income ($),Household net wealth ($),Labour market insecurity (%),Employment rate (%),Long-term unemployment rate (%),Personal earnings ($),...,Water quality (%),Stakeholder engagement for developing regulations (average score),Voter turnout (%),Life expectancy (years),Self-reported health (%),Life satisfaction (average score),Feeling safe walking alone at night (%),Homicide rate (ratio),Employees working very long hours (%),Time devoted to leisure and personal care (hours)
0,Australia,NaN,19.4,NaN,37433,528768,3.1,73,1,55206,...,92,2.7,92,83,85,7.1,67,0.9,12.5,14.36
1,Austria,0.8,20.8,1.6,37001,309637,2.3,72,1.3,53132,...,92,1.3,76,82,71,7.2,86,0.5,5.3,14.51
2,Belgium,0.7,20,2.1,34884,447607,2.4,65,2.3,54327,...,79,2,88,82.1,74,6.8,56,1.1,4.3,15.52
3,Canada,0.2,22.9,2.6,34421,478240,3.8,70,0.5,55342,...,90,2.9,68,82.1,89,7,78,1.2,3.3,14.57
4,Chile,9.4,18.4,1.9,NaN,135787,7,56,NaN,26729,...,62,1.3,47,80.6,60,6.2,41,2.4,7.7,NaN
5,Colombia,12.3,NaN,1,NaN,NaN,NaN,58,1.1,NaN,...,82,1.4,53,76.7,80,5.7,50,23.1,23.7,NaN
6,Costa Rica,2.3,17,1.2,16517,NaN,NaN,55,1.5,NaN,...,87,1.8,66,80.5,73,6.3,47,10,22,NaN
7,Czech Republic,0.5,23.4,1.5,26664,NaN,2.3,74,0.6,29885,...,89,1.6,62,79.3,62,6.9,77,0.7,4.5,NaN
8,Denmark,0.5,23.3,1.9,33774,149864,4.5,74,0.9,58430,...,93,2,85,81.5,70,7.5,85,0.5,1.1,NaN
9,Estonia,5.7,17,1.7,23784,188627,5.4,74,1.2,30720,...,86,2.7,64,78.8,57,6.5,79,1.9,2.2,14.98


In [80]:
source.dtypes

Country                                                              object
Dwellings without basic facilities (%)                               object
Housing expenditure (%)                                              object
Rooms per person (ratio)                                             object
Household net adjusted disposable income ($)                         object
Household net wealth ($)                                             object
Labour market insecurity (%)                                         object
Employment rate (%)                                                  object
Long-term unemployment rate (%)                                      object
Personal earnings ($)                                                object
Quality of support network (%)                                       object
Educational attainment (%)                                           object
Student skills (average score)                                       object
Years in edu

### Indicator Dropdown

In [81]:
indicators = list(source.columns)[1:]

indicator_dropdown = alt.binding_select(options=indicators, name="Indicator: ")

indicator_param = alt.param(value=indicators[0], bind=indicator_dropdown)

### Color Scale with Higlisgh Click

In [82]:
click = alt.selection_point(encodings=['x'])

color_scale = alt.condition(
    click,
    alt.Color('y:Q', scale=alt.Scale(scheme='yellowgreenblue'), legend=None),
    alt.value('lightgray')
)

In [83]:
chart = alt.Chart(source).mark_bar().encode(
    x=alt.X('Country:N', sort='-y', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('y:Q', title=''),
    color=color_scale,
    tooltip='tooltip:N',
    opacity=alt.condition(click, alt.value(1), alt.value(0.7))
).properties(
    width=800,
    height=400
).transform_calculate(
    y=f'datum[{indicator_param.name}]' # Variable Y axis
).transform_calculate(
    tooltip='datum.Country + ": " + format(datum.y, ",")'
).transform_filter(
    'isValid(datum.y)' # Removes NaN values
).add_params(
    indicator_param # Add dropdown parameter
).add_params(
    click
)

chart
# chart.save('CoolChart.html')

c:\Users\Dom\Dev\Newcastle\CSC3833\W1\venv\lib\site-packages\altair\utils\core.py:410: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
